In [1]:
import keras
from keras.callbacks import *
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class SimpleOCR(keras.Model):

    def __init__(self, num_classes=10, use_bn=False, use_dp=False):
        super(SimpleOCR, self).__init__()
        self.use_bn = use_bn
        self.use_dp = use_dp
        self.num_classes = num_classes

        if self.use_dp:
            self.dp = keras.layers.Dropout(0.1)
        if self.use_bn:
            self.bn = keras.layers.BatchNormalization(axis=-1)

    def conv_pool(self, x, units, conv_kernel=(3, 3), pool_size=(2, 2), strides=(1, 1)):
        x = Conv2D(units, kernel_size=conv_kernel, padding='same', strides=strides, name=None)(x)
        x = Activation(activation='relu')(x)
        if self.use_bn:
            x = self.bn(x)
        x = MaxPool2D(pool_size=pool_size)(x)
        if self.use_dp:
            x = self.dp(x)
        return x

    def dense(self, x, units, kernel_initializer='he_normal', acti='relu', laye='mid'):
        x = Dense(units, kernel_initializer=kernel_initializer)(x)
        if laye == 'mid' and self.use_bn:
            x = self.bn(x)
        x = Activation(acti)(x)
        if laye == 'mid' and self.use_dp:
            x = self.dp(x)
        return x

    def call(self, inputs, mask=None):
        print('calling call')
        x = self.conv_pool(inputs, 32)
        x = self.conv_pool(x, 64)
        x = self.conv_pool(x, 512)

        x = Flatten()(x)
        x = self.dense(x, 1000, laye='mid')
        x = self.dense(x, 10, laye='end')

        return x

In [6]:
datagen = image.ImageDataGenerator(featurewise_center=False,
                                   samplewise_center=False,
                                   featurewise_std_normalization=False,
                                   samplewise_std_normalization=False,
                                   zca_whitening=False,
                                   rotation_range=0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.,
                                   zoom_range=0.2,
                                   channel_shift_range=0.,
                                   fill_mode='nearest',
                                   cval=0.0,
                                   horizontal_flip=False,
                                   vertical_flip=False,
                                   rescale=1. / 255,
                                   preprocessing_function=None,
                                   # data_format=K.image_data_format(),
                                   )

train_generator = datagen.flow_from_directory(
    r'F:\number_ok1',  # this is the target directory
    target_size=(48, 48),  # all images will be resized to 48*40
    batch_size=256,
    class_mode='categorical',
    color_mode='grayscale')

validation_generator = datagen.flow_from_directory(
    r'F:\number_ok1',
    target_size=(48, 48),
    batch_size=128,
    class_mode='categorical',
    color_mode='grayscale')

model = SimpleOCR(use_dp=True)
print(model.layers)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print('\n'.join([str(tmp) for tmp in model.layers]))
print('model length: %s' % len(model.layers))

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
model.fit_generator(
    train_generator,
    steps_per_epoch=512,
    epochs=20,
    validation_data=validation_generator,
    nb_val_samples=100,
    verbose=True,
    callbacks=[early_stopping])

model.save('cnn3_gen_1.5.h5')

Found 123520 images belonging to 10 classes.
Found 123520 images belonging to 10 classes.
model length: 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=512, epochs=20, validation_data=<keras_pre..., verbose=True, callbacks=[<keras.ca..., validation_steps=100)`


RuntimeError: You must compile your model before using it.

In [60]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import os

In [83]:
irs = tf.InteractiveSession()

In [97]:
width = 14
height = 14
feat_stride = 16
shift_x = tf.range(width) * feat_stride  # width
shift_y = tf.range(height) * feat_stride  # height
shift_x, shift_y = tf.meshgrid(shift_x, shift_y)

In [99]:
sx = tf.reshape(shift_x, shape=(-1,))
sy = tf.reshape(shift_y, shape=(-1,))

In [111]:
shifts = tf.transpose(tf.stack([sx, sy, sx, sy]))
print(shifts.eval().shape)

K = tf.multiply(width, height)
print(K.eval())

shifts = tf.transpose(tf.reshape(shifts, shape=[1, K, 4]), perm=(1, 0, 2))
shifts.eval().shape

(196, 4)
196


(196, 1, 4)

---

In [112]:
base_size=16
ratios=[0.5, 1, 2]
scales=2 ** np.arange(3, 6)

In [114]:
base_anchor = np.array([1, 1, base_size, base_size]) - 1
print(base_anchor)

[ 0  0 15 15]


In [147]:
def _whctrs(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """
    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    print('w h xcenter ycenter', w, h, x_ctr, y_ctr)
    return w, h, x_ctr, y_ctr

def _mkanchors(ws, hs, x_ctr, y_ctr):
    """
    Given a vector of widths (ws) and heights (hs) around a center
    (x_ctr, y_ctr), output a set of anchors (windows).
    """

    ws = ws[:, np.newaxis]
    hs = hs[:, np.newaxis]
    
    anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                       y_ctr - 0.5 * (hs - 1),
                       x_ctr + 0.5 * (ws - 1),
                       y_ctr + 0.5 * (hs - 1)))
    return anchors

def _ratio_enum(anchor, ratios):
    """
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """
    print('origin anchor', anchor)
    w, h, x_ctr, y_ctr = _whctrs(anchor)
    size = w * h
    print('size', size)
    size_ratios = size / ratios
    print('size_ratios', size_ratios)
    ws = np.round(np.sqrt(size_ratios))
    hs = np.round(ws * ratios)
    print('ws', ws)
    print('hs', hs)
    
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors

def _scale_enum(anchor, scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """
    print('origin anchor', anchor)
    w, h, x_ctr, y_ctr = _whctrs(anchor)
    ws = w * scales
    hs = h * scales
    print('ws', ws)
    print('hs', hs)
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors

In [148]:
ratio_anchors = _ratio_enum(base_anchor, ratios)
print(ratio_anchors)

origin anchor [ 0  0 15 15]
w h xcenter ycenter 16 16 7.5 7.5
size 256
size_ratios [512. 256. 128.]
ws [23. 16. 11.]
hs [12. 16. 22.]
[[-3.5  2.  18.5 13. ]
 [ 0.   0.  15.  15. ]
 [ 2.5 -3.  12.5 18. ]]


In [149]:
scales

array([ 8, 16, 32], dtype=int32)

In [150]:
_scale_enum(ratio_anchors[0, :], scales)

origin anchor [-3.5  2.  18.5 13. ]
w h xcenter ycenter 23.0 12.0 7.5 7.5
ws [184. 368. 736.]
hs [ 96. 192. 384.]


array([[ -84.,  -40.,   99.,   55.],
       [-176.,  -88.,  191.,  103.],
       [-360., -184.,  375.,  199.]])

In [154]:
anchors = np.vstack([_scale_enum(ratio_anchors[i, :], scales) for i in range(ratio_anchors.shape[0])])

origin anchor [-3.5  2.  18.5 13. ]
w h xcenter ycenter 23.0 12.0 7.5 7.5
ws [184. 368. 736.]
hs [ 96. 192. 384.]
origin anchor [ 0.  0. 15. 15.]
w h xcenter ycenter 16.0 16.0 7.5 7.5
ws [128. 256. 512.]
hs [128. 256. 512.]
origin anchor [ 2.5 -3.  12.5 18. ]
w h xcenter ycenter 11.0 22.0 7.5 7.5
ws [ 88. 176. 352.]
hs [176. 352. 704.]


In [152]:
shifts.shape

TensorShape([Dimension(196), Dimension(1), Dimension(4)])

In [155]:
A = anchors.shape[0]
anchor_constant = tf.constant(anchors.reshape((1, A, 4)), dtype=tf.int32)

In [156]:
anchor_constant.shape

TensorShape([Dimension(1), Dimension(9), Dimension(4)])

In [161]:
length = K * A
anchors_tf = tf.reshape(tf.add(anchor_constant, shifts), shape=(length, 4))
anchors_tf.eval().shape

(1764, 4)

In [170]:
tf.concat(axis=0, values=[[1, 2, -1], [14]])[0].eval()

1

In [166]:
anchors

array([[ -84.,  -40.,   99.,   55.],
       [-176.,  -88.,  191.,  103.],
       [-360., -184.,  375.,  199.],
       [ -56.,  -56.,   71.,   71.],
       [-120., -120.,  135.,  135.],
       [-248., -248.,  263.,  263.],
       [ -36.,  -80.,   51.,   95.],
       [ -80., -168.,   95.,  183.],
       [-168., -344.,  183.,  359.]])

In [172]:
anchors.ravel()

array([ -84.,  -40.,   99.,   55., -176.,  -88.,  191.,  103., -360.,
       -184.,  375.,  199.,  -56.,  -56.,   71.,   71., -120., -120.,
        135.,  135., -248., -248.,  263.,  263.,  -36.,  -80.,   51.,
         95.,  -80., -168.,   95.,  183., -168., -344.,  183.,  359.])

In [173]:
anchors.ravel().argsort()

array([ 8, 33, 21, 20,  9,  4, 32, 29, 17, 16,  5,  0, 28, 25, 12, 13,  1,
       24, 26,  3, 14, 15, 30, 27,  2,  7, 18, 19, 31, 34,  6, 11, 23, 22,
       35, 10], dtype=int64)

In [174]:
anchors.ravel().argsort()[::-1]

array([10, 35, 22, 23, 11,  6, 34, 31, 19, 18,  7,  2, 27, 30, 15, 14,  3,
       26, 24,  1, 13, 12, 25, 28,  0,  5, 16, 17, 29, 32,  4,  9, 20, 21,
       33,  8], dtype=int64)

In [ ]:
anchors.ravel()[:, np.newaxis]

---

In [2]:
from keras import backend as K

In [408]:
x = Input((1, 10))
l = Dense(10)(x)
# l_ = Dense(10)(l)
ll = Dropout(0.1)(l)
model = Model(x, ll)

model.layers[1].set_weights((np.ones([10,10]), np.zeros([10])))

layer = K.function([model.layers[1].input, K.learning_phase()], [model.layers[2].output])

# output in test mode = 0
layer_output = layer([np.ones((1, 1, 10)), 1])
layer_output

[array([[[11.111112, 11.111112, 11.111112,  0.      , 11.111112,
          11.111112, 11.111112, 11.111112, 11.111112, 11.111112]]],
       dtype=float32)]

In [409]:
10 / 0.9

11.11111111111111

In [364]:
np.dot(np.ones((1, 1, 10)), model.layers[1].get_weights()[0]) + model.layers[1].get_weights()[1]

array([[[11., 11., 11., 11., 11., 11., 11., 11., 11., 11.]]])

In [386]:
model.layers[1].set_weights((np.ones([10,10]), np.zeros([10])))

In [395]:
model.layers[1].get_weights()

[array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]